In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Practicum/v1/train_df.csv')

Define the features 'X' and target variable 'y'

In [ ]:
X = df.drop('TOTALPMT_ADJ',axis=1).drop('LOG_TOTALPMT', axis=1)
y = df[['TOTALPMT_ADJ']]

In [ ]:
X

,LICNSTAT_AK,LICNSTAT_AL,LICNSTAT_AR,LICNSTAT_AZ,LICNSTAT_CA,LICNSTAT_CO,LICNSTAT_CT,LICNSTAT_DC,LICNSTAT_DE,LICNSTAT_FL,...,OUTCOME_Cannot Be Determined from Available Records,OUTCOME_Death,OUTCOME_Emotional Injury Only,OUTCOME_Insignificant Injury,OUTCOME_Major Permanent Injury,OUTCOME_Major Temporary Injury,OUTCOME_Minor Permanent Injury,OUTCOME_Minor Temporary Injury,"OUTCOME_Quadriplegic, Brain Damage, Lifelong Care",OUTCOME_Significant Permanent Injury
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
161134,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
161135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
161136,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


I. Hyperparameter Tuning

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
mlp = MLPRegressor(max_iter=200, early_stopping=True)

In [ ]:
search_space = {
    'hidden_layer_sizes': [5, 10], 
    'alpha': [0.0001, 0.001]
}

In [ ]:
GS_object = GridSearchCV(
    mlp, 
    param_grid = search_space,
    scoring = 'neg_root_mean_squared_error',
    cv = 3, # cv = k-fold cross validation. The input is k.
    verbose = 4,
    n_jobs= -1)

In [ ]:
# again, gridsearch will automatically conduct k-fold and determine the optimal parameters using k train and validation sets
GS_object.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, b

GridSearchCV(cv=3, estimator=MLPRegressor(early_stopping=True), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001],
                         'hidden_layer_sizes': [5, 10]},
             scoring='neg_root_mean_squared_error', verbose=4)

Save to csv

In [ ]:
gs_df = pd.DataFrame(GS_object.cv_results_)
gs_df.to_csv('/content/drive/MyDrive/Practicum/v1/gs_df_mlp.csv')

III. Model Evaluation

Obtain Optimal Hyperparameters

In [ ]:
GS_object.best_params_

{'alpha': 0.0001, 'hidden_layer_sizes': 10}

Define the KFold splits

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
n_splits = 5
cv = KFold(n_splits)

In [ ]:
mlp_optim = MLPRegressor( 
    hidden_layer_sizes= GS_object.best_params_.get('hidden_layer_sizes'), 
    activation='relu',
    solver='adam',
    alpha = GS_object.best_params_.get('alpha'),
    learning_rate = 'constant',
    learning_rate_init = 0.001,
    max_iter = 300,
    early_stopping=True
    )

Evaluate Errors

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
errors = []
for train_index, test_index in cv.split(X):
    X_train_kfold, y_train_kfold = X.iloc[train_index], y.iloc[train_index]
    X_test_kfold, y_test_kfold = X.iloc[test_index], y.iloc[test_index]
    mlp_optim.fit(X_train_kfold, y_train_kfold)
    y_pred = mlp_optim.predict(X_test_kfold)
    errors.append(np.sqrt(mean_squared_error(y_test_kfold, y_pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, b

In [ ]:
errors 

[261637.9309224144,
 269475.79050049215,
 262130.96203249734,
 259725.62088146273,
 262474.88955083914]

In [ ]:
np.mean(errors)

263089.03877754114

In [ ]:
er_df = pd.DataFrame({'errors': errors})
er_df.to_csv('/content/drive/MyDrive/Practicum/v1/er_df_mlp.csv')

Final Evaluation Test

In [ ]:
X

,LICNSTAT_AK,LICNSTAT_AL,LICNSTAT_AR,LICNSTAT_AZ,LICNSTAT_CA,LICNSTAT_CO,LICNSTAT_CT,LICNSTAT_DC,LICNSTAT_DE,LICNSTAT_FL,...,OUTCOME_Cannot Be Determined from Available Records,OUTCOME_Death,OUTCOME_Emotional Injury Only,OUTCOME_Insignificant Injury,OUTCOME_Major Permanent Injury,OUTCOME_Major Temporary Injury,OUTCOME_Minor Permanent Injury,OUTCOME_Minor Temporary Injury,"OUTCOME_Quadriplegic, Brain Damage, Lifelong Care",OUTCOME_Significant Permanent Injury
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
161134,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
161135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
161136,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Practicum/v1/test_df.csv')

In [ ]:
X_test = test_df.drop('TOTALPMT_ADJ',axis=1).drop('LOG_TOTALPMT', axis=1)
y_test = test_df[['TOTALPMT_ADJ']]

In [ ]:
X

,LICNSTAT_AK,LICNSTAT_AL,LICNSTAT_AR,LICNSTAT_AZ,LICNSTAT_CA,LICNSTAT_CO,LICNSTAT_CT,LICNSTAT_DC,LICNSTAT_DE,LICNSTAT_FL,...,OUTCOME_Cannot Be Determined from Available Records,OUTCOME_Death,OUTCOME_Emotional Injury Only,OUTCOME_Insignificant Injury,OUTCOME_Major Permanent Injury,OUTCOME_Major Temporary Injury,OUTCOME_Minor Permanent Injury,OUTCOME_Minor Temporary Injury,"OUTCOME_Quadriplegic, Brain Damage, Lifelong Care",OUTCOME_Significant Permanent Injury
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
161134,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
161135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
161136,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
X_test

,LICNSTAT_AK,LICNSTAT_AL,LICNSTAT_AR,LICNSTAT_AZ,LICNSTAT_CA,LICNSTAT_CO,LICNSTAT_CT,LICNSTAT_DC,LICNSTAT_DE,LICNSTAT_FL,...,OUTCOME_Cannot Be Determined from Available Records,OUTCOME_Death,OUTCOME_Emotional Injury Only,OUTCOME_Insignificant Injury,OUTCOME_Major Permanent Injury,OUTCOME_Major Temporary Injury,OUTCOME_Minor Permanent Injury,OUTCOME_Minor Temporary Injury,"OUTCOME_Quadriplegic, Brain Damage, Lifelong Care",OUTCOME_Significant Permanent Injury
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40280,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
40281,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
40282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
40283,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
mlp_optim.fit(X, y)
y_pred = mlp_optim.predict(X_test)
mean_squared_error(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, b

67999502030.67736

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

260767.14139376793